### < 모델 생성 후 고려해볼 것들>
1. 모델을 여러가지 사용해보기 ex) 앙상블, XGBoost, Light GBM, LSTM
2. 이상치를 중앙값으로 할지, 최대치로 할지
3. 시계열 데이터를 어떻게 처리할지 ex) 4분기, 2분기 등
4. 판매량, 판매 가격의 데이터 쏠림을 전처리 할지 말지
-> 팀당 하루에 5번 제출 가능

In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from os.path import join

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# 1. 데이터 불러오기

In [2]:
data_dir = os.getenv('HOME') + '/kaggle/predict_future_sales/data'

items_path = os.path.join(data_dir, 'items.csv')
item_category_path = os.path.join(data_dir, 'item_categories.csv')
shops_path = os.path.join(data_dir, 'shops.csv')
train_path = os.path.join(data_dir, 'sales_train.csv')
test_path = os.path.join(data_dir, 'test.csv')

In [3]:
items = pd.read_csv(items_path)
categories = pd.read_csv(item_category_path)
shops = pd.read_csv(shops_path)
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

<br><br><br>

# 2. 데이터 분석

In [4]:
data = pd.read_csv(train_path)
sub = pd.read_csv(test_path)
print('train data dim : {}'.format(data.shape))
print('test data dim : {}'.format(sub.shape))

train data dim : (2935849, 6)
test data dim : (214200, 3)


학습데이터는 100만개, 테스트 데이터는 21만개로 이루어져 있음. 
test 데이터의 컬럼이 3개인것을 볼 수 있는데, 제외된 3개의 컬럼 중 하나는 우리가 마주어야 할 집의 가격 `price`일 것임

In [5]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [7]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [8]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [9]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


## 3. Feature Engineering

#### 3.1 데이터 결측치 확인

In [11]:
train.isnull().sum()

date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

#### 3.2 중복 제거

In [12]:
sum(train.duplicated()), sum(test.duplicated())

(6, 0)

In [13]:
train[train.duplicated() == True]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
76962,05.01.2013,0,54,20130,149.0,1.0
1435367,23.02.2014,13,50,3423,999.0,1.0
1496766,23.03.2014,14,21,3423,999.0,1.0
1671873,01.05.2014,16,50,3423,999.0,1.0
1866340,12.07.2014,18,25,3423,999.0,1.0
2198566,31.12.2014,23,42,21619,499.0,1.0


In [14]:
train.drop_duplicates(inplace=True)

In [15]:
#중복 재확인
sum(train.duplicated())

0

#### 3.3 아이템 대분류 만들기(은지님 추가)

#### 3.4 지역별 가게 대분류 만들기

In [16]:
shop_data_path = join(data_dir, 'shops.csv')
shops = pd.read_csv(shop_data_path)
pd.DataFrame(shops.shop_name.values.reshape(-1, 4))

,0,1,2,3
0,"!Якутск Орджоникидзе, 56 фран","!Якутск ТЦ ""Центральный"" фран","Адыгея ТЦ ""Мега""","Балашиха ТРК ""Октябрь-Киномир"""
1,"Волжский ТЦ ""Волга Молл""","Вологда ТРЦ ""Мармелад""","Воронеж (Плехановская, 13)","Воронеж ТРЦ ""Максимир"""
2,"Воронеж ТРЦ Сити-Парк ""Град""",Выездная Торговля,Жуковский ул. Чкалова 39м?,Жуковский ул. Чкалова 39м²
3,Интернет-магазин ЧС,"Казань ТЦ ""Бехетле""","Казань ТЦ ""ПаркХаус"" II","Калуга ТРЦ ""XXI век"""
4,"Коломна ТЦ ""Рио""","Красноярск ТЦ ""Взлетка Плаза""","Красноярск ТЦ ""Июнь""","Курск ТЦ ""Пушкинский"""
5,"Москва ""Распродажа""","Москва МТРЦ ""Афи Молл""",Москва Магазин С21,"Москва ТК ""Буденовский"" (пав.А2)"
6,"Москва ТК ""Буденовский"" (пав.К7)","Москва ТРК ""Атриум""","Москва ТЦ ""Ареал"" (Беляево)","Москва ТЦ ""МЕГА Белая Дача II"""
7,"Москва ТЦ ""МЕГА Теплый Стан"" II","Москва ТЦ ""Новый век"" (Новокосино)","Москва ТЦ ""Перловский""","Москва ТЦ ""Семеновский"""
8,"Москва ТЦ ""Серебряный Дом""","Мытищи ТРК ""XL-3""","Н.Новгород ТРЦ ""РИО""","Н.Новгород ТРЦ ""Фантастика"""
9,"Новосибирск ТРЦ ""Галерея Новосибирск""","Новосибирск ТЦ ""Мега""","Омск ТЦ ""Мега""","РостовНаДону ТРК ""Мегацентр Горизонт"""


In [17]:
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops.head(10)

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский
5,"Вологда ТРЦ ""Мармелад""",5,Вологда
6,"Воронеж (Плехановская, 13)",6,Воронеж
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж
9,Выездная Торговля,9,Выездная


In [18]:
# 문자 데이터를 모두 수치 데이터로 Encoding
shops.city.value_counts()

Москва              13
Якутск               4
Тюмень               3
РостовНаДону         3
Воронеж              3
Новосибирск          2
Казань               2
СПб                  2
Уфа                  2
Самара               2
Н.Новгород           2
Красноярск           2
Жуковский            2
Чехов                1
Мытищи               1
Интернет-магазин     1
Волжский             1
Химки                1
Адыгея               1
Сургут               1
Томск                1
Ярославль            1
Цифровой             1
Курск                1
Омск                 1
Сергиев              1
Балашиха             1
Калуга               1
Коломна              1
Выездная             1
Вологда              1
Name: city, dtype: int64

In [19]:
from sklearn.preprocessing import LabelEncoder
city_encoder = LabelEncoder()
shops['city_code'] = city_encoder.fit_transform(shops['city'])

In [20]:
shops.head(10)

,shop_name,shop_id,city,city_code
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск,29
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск,29
2,"Адыгея ТЦ ""Мега""",2,Адыгея,0
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,1
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,2
5,"Вологда ТРЦ ""Мармелад""",5,Вологда,3
6,"Воронеж (Плехановская, 13)",6,Воронеж,4
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж,4
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж,4
9,Выездная Торговля,9,Выездная,5


#### train 데이터에 city_code, item_code 추가하기

In [21]:
merged_df = pd.merge(train, shops, on="shop_id")
merged_df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,city,city_code
0,02.01.2013,0,59,22154,999.0,1.0,"Ярославль ТЦ ""Альтаир""",Ярославль,30
1,10.01.2013,0,59,22151,399.0,1.0,"Ярославль ТЦ ""Альтаир""",Ярославль,30
2,04.01.2013,0,59,5603,699.0,1.0,"Ярославль ТЦ ""Альтаир""",Ярославль,30
3,19.01.2013,0,59,5587,199.0,2.0,"Ярославль ТЦ ""Альтаир""",Ярославль,30
4,31.01.2013,0,59,5613,5571.0,1.0,"Ярославль ТЦ ""Альтаир""",Ярославль,30
...,...,...,...,...,...,...,...,...,...
2935838,30.10.2015,33,36,20231,169.0,1.0,"Новосибирск ТРЦ ""Галерея Новосибирск""",Новосибирск,16
2935839,18.10.2015,33,36,20334,1949.0,1.0,"Новосибирск ТРЦ ""Галерея Новосибирск""",Новосибирск,16
2935840,31.10.2015,33,36,20339,1549.0,1.0,"Новосибирск ТРЦ ""Галерея Новосибирск""",Новосибирск,16
2935841,31.10.2015,33,36,20352,1549.0,1.0,"Новосибирск ТРЦ ""Галерея Новосибирск""",Новосибирск,16


In [25]:
merged_df.drop(['shop_name','city'], axis=1, inplace=True)
merged_df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,city_code
0,02.01.2013,0,59,22154,999.0,1.0,30
1,10.01.2013,0,59,22151,399.0,1.0,30
2,04.01.2013,0,59,5603,699.0,1.0,30
3,19.01.2013,0,59,5587,199.0,2.0,30
4,31.01.2013,0,59,5613,5571.0,1.0,30
...,...,...,...,...,...,...,...
2935838,30.10.2015,33,36,20231,169.0,1.0,16
2935839,18.10.2015,33,36,20334,1949.0,1.0,16
2935840,31.10.2015,33,36,20339,1549.0,1.0,16
2935841,31.10.2015,33,36,20352,1549.0,1.0,16


#### 3.4 이상치 제거(진오님)